# GSPE Example File
This notebook runs HOPP and applies the correct power electronics efficency factors established in the Green Steel Power Electronics (GSPE) simulator

NOTE: all of the economic analysis will use the efficency factors that are default in PySAM/set in HOPP. The goal of this script is to solely return a power time series with teh correct efficency factors.

In [5]:
from hopp.simulation import HoppInterface
import numpy as np

hi = HoppInterface("./inputs/08-GSPE.yaml")
hi.simulate(30)

print("simulation complete")

FLORIS is the system model...
Simulating wind farm output in FLORIS...
Wind annual energy:  71702641.3705228
simulation complete


## Work on implementing new power electronic efficencies according to the GSPE code

1) revert efficency factors applied in HOPP

In [9]:
gen, annual_energy, capacity_factor = dict(), dict(),  dict()

# wind
wm = hi.hopp.system.technologies['wind']._system_model

if hi.hopp.system.technologies['wind'].config.model_name == 'floris':
    DEFAULT_PYSAM_LOSSES = 12.83
    gen['wind'] = wm.gen * ((100 - DEFAULT_PYSAM_LOSSES)/100) ** -1 # revert PySAM default losses
    annual_energy['wind'] = np.sum(gen['wind'])
    capacity_factor['wind'] = annual_energy['wind'] / (8760 * wm.system_capacity) * 100

else: # wm.config.model_name == 'pysam'
    non_wake_losses = 0
    pysam_losses_dict = wm.export()['Losses']

    relevant_losses = ['avail_bop_loss', 'avail_grid_loss', 'avail_turb_loss', 'elec_eff_loss', 'elec_parasitic_loss', 
                        'env_degrad_loss', 'env_env_loss', 'env_exposure_loss', 'env_icing_loss', 'ops_env_loss', 
                        'ops_grid_loss', 'ops_load_loss', 'ops_strategies_loss']
    
    for losses in relevant_losses:
        non_wake_losses += pysam_losses_dict[losses]
    
    gen['wind'] = [i * ((100 - non_wake_losses)/100)**-1 for i in wm.Outputs.gen][:8760]
    annual_energy['wind'] = np.sum(gen['wind'])
    capacity_factor['wind'] = annual_energy['wind'] / (8760 * wm.Farm.system_capacity) * 100

# solar
pvm = hi.hopp.system.technologies['pv']._system_model

pysam_losses = ((100 - pvm.SystemDesign.inv_eff) + pvm.SystemDesign.losses) / 100

gen['pv'] = [i * 1e-3 for i in pvm.Outputs.dc] # pvm.Outputs.dc in W, convert to kW (dc power should be without losses applied)
annual_energy['pv'] = np.sum(gen['pv']) # hourly time series, result is in kWh
capacity_factor['pv'] = annual_energy['pv'] / (8760 * pvm.SystemDesign.system_capacity) * 100 # system capacity in kWh

# battery
# TODO what efficencies do I revert for battery?
battery = hi.hopp.system.technologies['battery']

gen['battery'] = battery.generation_profile
annual_energy['battery'] = np.sum(gen['battery']) # same as battery.annual_energy_kwh... 
capacity_factor['battery'] = annual_energy['battery'] / (8760 * battery.system_capacity_kw) * 100

print('done')


done


2) calculate losses according to GSPE code

In [8]:
eff = {'back_to_back': 0.9604,
       'diode_based_rectifier': 0.96,
       'thyristor_based_rectifier': 0.992,
       'IGBT_based_inverter': 0.98,
       'bi_DCDC_converter': 0.985,
       'uni_DCDC_converter': 0.985,
       'transformer_up': 0.99,
       'transformer_down': 0.99,
       'rectifier_power_factor': 0.89,
       'transformer_power_factor': 0.85,
       }

# TODO instead of "gutting" the code, could package the file and then use as module
DFIG_AC_1_eff_wind = eff['transformer_up'] * eff['transformer_down'] * eff['diode_based_rectifier'] # TODO excluding battery losses here... is that what I am supposed to do?
DFIG_AC_1_eff_solar = eff['IGBT_based_inverter'] * eff['transformer_down'] * eff['diode_based_rectifier'] 
DFIG_AC_1_eff_battery = eff['bi_DCDC_converter'] # assuming bi-directional... is this right?

efficencies = {'wind': DFIG_AC_1_eff_wind, 'pv': DFIG_AC_1_eff_solar, 'battery': DFIG_AC_1_eff_battery}

# apply each of the efficencies
for d in [gen, annual_energy, capacity_factor]:
       for k in d.keys(): 
              if type(d[k]) == np.float64:
                     d[k] = d[k] * efficencies[k]
              else:
                     d[k] = [i * efficencies[k] for i in d[k]]


print('done')
# TODO apply the above efficency to the powers in the HI
# TODO this is done after the hopp simulaiton is complete, meaning any 
# costs assoicated with the power generation are going to have to be recalculated...
# not sure if there is a better way of doing this without completely changing HOPP



KeyboardInterrupt: 